## 千兆病理图像癌细胞转移检测
## Detecting Cancer Metastases on Gigapixel Pathology Images

In [1]:
import openslide
import numpy as np
from pylab import *

# 读取包含有肿瘤区域的大图（全切片病理图像）
origin_images_path = "/atlas/home/zwpeng/paper_rebuild/camelyon/train/tumor/origin_images/Tumor_005.tif"
origin_slide = openslide.open_slide(origin_images_path)

# 读取该肿瘤区域的标注图
annotation_images_path = "/atlas/home/zwpeng/paper_rebuild/camelyon/train/tumor/annotation_images/Tumor_005_Mask.tif"
mask_slide = openslide.open_slide(annotation_images_path)

In [2]:
# origin_slide.level_count,origin_slide.level_dimensions,origin_slide.level_downsamples     #查看病理图片的金字塔结构

In [3]:
# mask_slide.level_count,mask_slide.level_dimensions,mask_slide.level_downsamples     #查看标注图片的金字塔结构

### 首先找到感兴趣区域

In [2]:
# 方法三：通过获取每一块区域的像素值R G B各自的平均数，然后相减，设置一个阈值，将噪点（墨迹）和有效区　分开

from pylab import *
import numpy as np

# 感兴趣区域锁定函数
def locate_ROI(origin_slide,level=6):

    origin_widths,origin_heights = origin_slide.dimensions

    object_widths,object_heights = origin_slide.level_dimensions[level]

    rgb_list_y = list()
    rgb_list_x = list()
    rgb_var_x = []
    rgb_var_y = []
    rgb_var_xi = []
    rgb_var_yi = []

    # 寻找有效区域的y值、高度
    for k in range(100):
        slide = origin_slide.read_region((0, k*origin_heights//100), level, (object_widths, object_heights//50)) 
        slide_arr = array(slide.convert("RGB"))
        arrR = np.mean(slide_arr[:,:,:1])
        arrG = np.mean(slide_arr[:,:,1:2])
        arrB = np.mean(slide_arr[:,:,2:3])
        rgb_list_y.append((arrR,arrG,arrB))
    for i,rgbVar in enumerate(rgb_list_y):
        rgb_var_y.append(np.var(rgbVar))
        if np.var(rgbVar)>=1:
            rgb_var_yi.append(i)

#     print(rgb_var_yi)
    effective_y = min(rgb_var_yi)*origin_heights//100        #有效区域的左上顶点y坐标找到了
    effective_heights = (max(rgb_var_yi)-min(rgb_var_yi))*origin_heights//100 + origin_heights//50  #有效区域的高度也出来了
#     print("有效区域的ｙ值是：%d" %effective_y, "有效区域的高度是：%d" %effective_heights)

    # 寻找有效区域的x值、宽度
    for j in range(100):
        slide = origin_slide.read_region((j*origin_widths//100, effective_y), level, 
                                          (object_widths//50, effective_heights//62))     # 循环顺序读取50宽的区域
    #     slide = origin_slide.read_region((j*origin_widths//100, 0), level, 
    #                                       (object_widths//50, object_heights))     # 循环顺序读取50宽的区域

        slide_arr = array(slide.convert("RGB"))
        arrR = np.mean(slide_arr[:,:,:1])
        arrG = np.mean(slide_arr[:,:,1:2])
        arrB = np.mean(slide_arr[:,:,2:3])
        rgb_list_x.append((arrR,arrG,arrB))
    for i,rgbVar in enumerate(rgb_list_x):
        rgb_var_x.append(np.var(rgbVar))
        if np.var(rgbVar)>=2:
            rgb_var_xi.append(i)

#     print(rgb_var_xi)
    effective_x = min(rgb_var_xi)*origin_widths//100        # 有效区域的左上顶点y坐标找到了
    effective_widths = (max(rgb_var_xi) - min(rgb_var_xi))*origin_widths//100 + origin_widths//50  # 有效区域的宽度也出来了
#     print("有效区域的ｘ值是：%d" %effective_x, "有效区域的宽度是：%d" %effective_widths)
#     plt.plot(range(100), rgb_var_y[:100], label='rgb_var_curve')
    # plt.plot(range(100), rgb_var_x[:100], label='rgb_var_curve')
#     plt.legend()
#     plt.show()
    return effective_x,effective_y,effective_widths,effective_heights

In [3]:
effective_x,effective_y,effective_widths,effective_heights = locate_ROI(origin_slide)

In [4]:
effective_x,effective_y,effective_widths,effective_heights

(18580, 125199, 41071, 50518)

这个有效区域，经过手动调试，起点坐标：(17600,124700) ，区域长宽：(44800,57600)



### mask 的有效区域，定位比较容易

In [5]:
from pylab import *
import numpy as np

def locate_ROI_mask(mask_slide,mask_level=7):

    # level0　的尺寸
    mask_widths, mask_heights = mask_slide.dimensions
    # level7 的尺寸
    mask_level_widths, mask_level_heights = mask_slide.level_dimensions[mask_level]

    mask_level_slide = mask_slide.read_region((0, 0), mask_level, (mask_level_widths, mask_level_heights))
    mask_level_slide_gray = mask_level_slide.convert("L")
    mask_level_slide_arr = array(mask_level_slide_gray)

    mask_y, mask_x = nonzero(mask_level_slide_arr)  # 因为mask是黑白图，只需直接获得非零像素的坐标
    # mask_x, mask_y
    tumor_leftup_x = (min(mask_x)-1) * int(mask_slide.level_downsamples[mask_level])
    tumor_leftup_y = (min(mask_y)-1) * int(mask_slide.level_downsamples[mask_level])
    tumor_rightdown_x = (max(mask_x)+1) * int(mask_slide.level_downsamples[mask_level])
    tumor_rightdown_y = (max(mask_y)+1) * int(mask_slide.level_downsamples[mask_level])
    
#     print(tumor_leftup_x,tumor_leftup_y,tumor_rightdown_x,tumor_rightdown_y)
    mask_effective_widths = tumor_rightdown_x - tumor_leftup_x
    mask_effective_heights = tumor_rightdown_y - tumor_leftup_y
    
#     mask_tumor_area = ((max(mask_x)-min(mask_x)+2)*int(mask_slide.level_downsamples[mask_level]), 
#                        (max(mask_y)-min(mask_y)+2)*int(mask_slide.level_downsamples[mask_level]))
#     print(mask_tumor_area)        # mask区域的长宽
    return tumor_leftup_x,tumor_leftup_y,mask_effective_widths,mask_effective_heights

In [6]:
tumor_leftup_x,tumor_leftup_y,mask_effective_widths,mask_effective_heights = locate_ROI_mask(mask_slide)

In [7]:
tumor_leftup_x,tumor_leftup_y,mask_effective_widths,mask_effective_heights

(57600, 136320, 2048, 3712)

In [10]:
# mask_slide.read_region((tumor_leftup_x,tumor_leftup_y),0,(mask_effective_widths,mask_effective_heights)) 

In [8]:
import numpy as np
from PIL.Image import Image
from pylab import *
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt

# 有效区域（感兴趣区域），所有细胞聚集区
# effective_area = (effective_x, effective_y)
# effective_area_size = (effective_widths, effective_heights)
# 有效区域，所有标注的细胞聚集区
# mask_slide.read_region(mask_tumor_start, 0, mask_tumor_area) 
# random_img = origin_slide.read_region((random_x,random_y),0,(widths,heights))  # 这一区域就是随机读到的区域，
                                                            #接下来就要判断这个区域在哪里，应该给他贴上什么标签

# 随机生成一个０到１之间的数，判断是否大于0.5,如果大于0.5,就从tumor区（mask）获取随机点（产生随机图片）；
# 如果小于0.5,就从normal区获取随机点，产生的随机图片需要判断是否取到了tumor
# for i in range(30):    # 这个其实就是 batch_size
#     widths, heights = 299, 299
def data_generator(widths=299,heights=299):
    while True:
        random_num = np.random.random(1)
        print("0到１之间的随机数是：%s"%random_num)

        if random_num > 0.5:
            # 定义随机坐标,一定要取到一张含有tumor的图片
            random_x = np.random.randint(tumor_leftup_x, tumor_leftup_x + mask_effective_widths - widths)  
            random_y = np.random.randint(tumor_leftup_y, tumor_leftup_y + mask_effective_heights - heights)
    #             print("取tumor随机点坐标是：%d,%d"%(random_x,random_y))
            random_img_mask = mask_slide.read_region((random_x,random_y),0,(widths,heights))
            random_img_mask_arr = array(random_img_mask.convert("L"))
            random__img_y, random_img_x = nonzero(random_img_mask_arr)
            while len(random_img_x)==0:
                random_x = np.random.randint(tumor_leftup_x, tumor_leftup_x + mask_effective_widths - widths)
                random_y = np.random.randint(tumor_leftup_y, tumor_leftup_y + mask_effective_heights - heights)
    #                 print("取tumor随机点坐标是：%d,%d"%(random_x,random_y))
                random_img_mask = mask_slide.read_region((random_x,random_y),0,(widths,heights))
                random_img_mask_arr = array(random_img_mask.convert("L"))
                random__img_y, random_img_x = nonzero(random_img_mask_arr)

            #*********************上面这个 while 循环结束后，就产生了一个合格的坐标点*********************#
            random_img = origin_slide.read_region((random_x,random_y),0,(widths,heights))
#             f,axes = plt.subplots(1,2)
#             plt.subplot(1,2,1)
#             plt.imshow(origin_slide.read_region((random_x,random_y),0,(widths,heights)))
#             plt.set_title('tumor_image')
#             plt.subplot(1,2,2)
#             plt.imshow(mask_slide.read_region((random_x,random_y),0,(widths,heights)))
#             plt.set_title('tumor_mask')
#             plt.show()

            #***接下来就给他贴标签，并处理成训练所需的数据结构***#
            random_img_arr = array(random_img.convert("RGB"))
            x = np.expand_dims(random_img_arr, axis=0)/255.
            y = to_categorical(0,2)    
        else:
            # 定义随机坐标，一定要取到一张不含有tumor的normal图片
            random_x = np.random.randint(effective_x,effective_x+effective_widths-widths)   # 大图上,nomal有效区的起点和终点
            random_y = np.random.randint(effective_y,effective_y+effective_heights-heights)
    #             print("取normal随机点坐标是：%d,%d"%(random_x,random_y))
            random_img_mask = mask_slide.read_region((random_x,random_y),0,(widths,heights))
            random_img_mask_arr = array(random_img_mask.convert("L"))
            random__img_y, random_img_x = nonzero(random_img_mask_arr)
            random_img = origin_slide.read_region((random_x,random_y),0,(widths,heights))

#             print("随机情况",len(random_img_x),(array(random_img.convert("RGB"))).std())

            while (array(random_img.convert("RGB"))).std()<20.0:
                random_x = np.random.randint(effective_x,effective_x+effective_widths-widths)
                random_y = np.random.randint(effective_y,effective_y+effective_heights-heights)
    #                 print("取normal随机点坐标是：%d,%d" %(random_x,random_y))
                random_img_mask = mask_slide.read_region((random_x,random_y),0,(widths,heights))
                random_img_mask_arr = array(random_img_mask.convert("L"))
                random__img_y, random_img_x = nonzero(random_img_mask_arr)
                random_img = origin_slide.read_region((random_x,random_y),0,(widths,heights))

#             print("颜色检测情况",len(random_img_x),(array(random_img.convert("RGB"))).std())

            while len(random_img_x) != 0:
                random_x = np.random.randint(effective_x,effective_x+effective_widths-widths)
                random_y = np.random.randint(effective_y,effective_y+effective_heights-heights)
    #                 print("取normal随机点坐标是：%d,%d" %(random_x,random_y))
                random_img_mask = mask_slide.read_region((random_x,random_y),0,(widths,heights))
                random_img_mask_arr = array(random_img_mask.convert("L"))
                random__img_y, random_img_x = nonzero(random_img_mask_arr)
                random_img = origin_slide.read_region((random_x,random_y),0,(widths,heights))

#             print("非tumor区检测情况",len(random_img_x), (array(random_img.convert("RGB"))).std())

            #*********************上面这个 while 循环结束后，就产生了一个合格的坐标点*********************#
            random_img = origin_slide.read_region((random_x,random_y),0,(widths,heights))
#             f,axes = plt.subplots(1,2)
#             plt.subplot(1,2,1)
#             plt.imshow(origin_slide.read_region((random_x,random_y),0,(widths,heights)))
#             plt.title('normal_image')
#             plt.subplot(1,2,2)
#             plt.imshow(mask_slide.read_region((random_x,random_y),0,(widths,heights)))
#             plt.title('normal_mask')
#             plt.show()
    #         f,axes = plt.subplots(1,2,figsize=(20,20))
    #         ax = axes.ravel()
    #         ax[0].imshow(origin_slide.read_region((random_x,random_y),0,(widths,heights)))
    #         ax[0].set_title('normal_image')
    #         ax[1].imshow(mask_slide.read_region((random_x,random_y),0,(widths,heights)))
    #         ax[1].set_title('normal_mask')
    #         f
            #***接下来就给他贴标签，并处理成训练所需的数据结构***#
            random_img_arr = array(random_img.convert("RGB"))
            x = np.expand_dims(random_img_arr, axis=0)/255.
            y = to_categorical(1,2) 
        yield (x,y)

Using TensorFlow backend.


In [23]:
a = data_generator()

In [33]:
a.__next__() 　　　　# yield要用next调用

0到１之间的随机数是：[ 0.79746816]


(array([[[[ 0.7254902 ,  0.42745098,  0.58039216],
          [ 0.74509804,  0.44705882,  0.59607843],
          [ 0.6745098 ,  0.39215686,  0.5372549 ],
          ..., 
          [ 0.25098039,  0.10980392,  0.25882353],
          [ 0.23921569,  0.09803922,  0.24705882],
          [ 0.25098039,  0.10980392,  0.25098039]],
 
         [[ 0.77254902,  0.4745098 ,  0.62352941],
          [ 0.7254902 ,  0.43529412,  0.58039216],
          [ 0.61176471,  0.3372549 ,  0.48235294],
          ..., 
          [ 0.21568627,  0.07843137,  0.24313725],
          [ 0.19607843,  0.07058824,  0.23137255],
          [ 0.20784314,  0.08235294,  0.23529412]],
 
         [[ 0.80784314,  0.50980392,  0.65882353],
          [ 0.70588235,  0.41568627,  0.56078431],
          [ 0.54901961,  0.28235294,  0.42745098],
          ..., 
          [ 0.18823529,  0.06666667,  0.23137255],
          [ 0.17647059,  0.05490196,  0.21568627],
          [ 0.19215686,  0.0745098 ,  0.22352941]],
 
         ..., 
         [

In [35]:
%time example_X, example_y  = next(data_generator())    # yield要用next调用

0到１之间的随机数是：[ 0.27024104]
CPU times: user 80.8 ms, sys: 36.9 ms, total: 118 ms
Wall time: 182 ms


In [38]:
example_X.shape,example_y.shape

((1, 299, 299, 3), (1, 2))

### 注意这里的第一个数表示batch_size


接下来可以开始训练了。

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, Activation, BatchNormalization
from keras.optimizers import SGD


def Creat_InvepV3(num_lay_out=64,classes=2,dropout=0.5):
    img_width, img_height = 299, 299
    base_model = InceptionV3(weights="imagenet", include_top=False,
                             input_shape=(img_width, img_height, 3))
    x = base_model.output
    x = Flatten()(x)
    x = Dense(num_lay_out)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout)(x)
    x = Dense(classes)(x)
    x = BatchNormalization()(x)
    x = Activation('softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer=SGD(lr=0.002, momentum=0.9, decay=0.5),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [11]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import SGD


def Creat_InvepV3(num_lay_out=1024,classes=2,dropout=0.5):
    img_width, img_height = 299, 299
    base_model = InceptionV3(weights="imagenet", include_top=False,
                             input_shape=(img_width, img_height, 3))
    x = base_model.output
    # x = Flatten()(x)
    # x = Dense(num_lay_out)(x)
    # x = BatchNormalization()(x)
    # x = Activation('relu')(x)
    # x = Dropout(dropout)(x)
    # x = Dense(classes)(x)
    # x = BatchNormalization()(x)
    # x = Activation('softmax')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_lay_out, activation='relu')(x)
    predictions = Dense(classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=SGD(lr=1e-4, momentum=0.9),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [12]:
from __future__ import division
import keras
from keras.callbacks import CSVLogger
import gc

def train_model():
    keras.backend.tensorflow_backend.clear_session()
    epochs = 100
    model = Creat_InvepV3()
    csvlogger = CSVLogger('training.log', append=True)
    model.fit_generator(data_generator(),
                        steps_per_epoch=100,
                        epochs=epochs,
                        validation_data=data_generator(),
                        validation_steps=20,
                        verbose=0,
                        callbacks=[csvlogger])
    model.save('model.h5')
    gc.collect()
train_model()

[ 0.69755674]
[ 0.31874213]
[ 0.92734423]
[ 0.59605501]
[ 0.00330573]
[ 0.20861872]
[ 0.73375468]
[ 0.74391351]
[ 0.08045171]
[ 0.76671148]
[ 0.7062153]
[ 0.0479816]
[ 0.38825133]
[ 0.3690825]
[ 0.10039305]
[ 0.90899472]
[ 0.3364072]
[ 0.04982857]
[ 0.61852767]
[ 0.59157018]
[ 0.12048484]
[ 0.62639401]
[ 0.82987909]
[ 0.16321608]
[ 0.26514581]
[ 0.49998707]
[ 0.84856679]
[ 0.28969771]
[ 0.2673296]
[ 0.75459928]
[ 0.6143002]
[ 0.1998602]
[ 0.75341666]
[ 0.08061198]
[ 0.85326269]
[ 0.84490313]
[ 0.15156037]
[ 0.17867301]
[ 0.85158782]
[ 0.21967199]
[ 0.03630589]
[ 0.86529792]
[ 0.64166248]
[ 0.43923919]
[ 0.6628592]
[ 0.5897882]
[ 0.48143153]
[ 0.54683358]
[ 0.94247645]
[ 0.95809271]
[ 0.67936129]
[ 0.37907165]
[ 0.14941705]
[ 0.70341351]
[ 0.95086578]
[ 0.98801629]
[ 0.41583115]
[ 0.9962518]
[ 0.57263351]
[ 0.64807627]
[ 0.62733418]
[ 0.95741483]
[ 0.37695576]
[ 0.75656603]
[ 0.81285693]
[ 0.41101085]
[ 0.91460179]
[ 0.4262937]
[ 0.79359301]
[ 0.3955277]
[ 0.53567543]
[ 0.93057791]
[ 0.

[ 0.94104604]
[ 0.32186382]
[ 0.65132853]
[ 0.67707482]
[ 0.03379142]
[ 0.74094183]
[ 0.59338978]
[ 0.80636804]
[ 0.55789725]
[ 0.59570334]
[ 0.40424573]
[ 0.88522129]
[ 0.8409416]
[ 0.9549993]
[ 0.10330691]
[ 0.27559479]
[ 0.43081115]
[ 0.84114556]
[ 0.89127773]
[ 0.91059716]
[ 0.53032913]
[ 0.58383677]
[ 0.99118896]
[ 0.03964318]
[ 0.44512046]
[ 0.58781259]
[ 0.94414145]
[ 0.72476943]
[ 0.25366677]
[ 0.96835086]
[ 0.44678618]
[ 0.28614482]
[ 0.79097332]
[ 0.63870942]
[ 0.10844977]
[ 0.5698464]
[ 0.8562814]
[ 0.60402958]
[ 0.44072398]
[ 0.41007428]
[ 0.83068489]
[ 0.42509355]
[ 0.36507033]
[ 0.78036371]
[ 0.50362605]
[ 0.30647257]
[ 0.66122082]
[ 0.95283238]
[ 0.56949605]
[ 0.73844564]
[ 0.83719763]
[ 0.62453768]
[ 0.53467529]
[ 0.60507126]
[ 0.23184241]
[ 0.17668533]
[ 0.42344736]
[ 0.0248918]
[ 0.28095761]
[ 0.29863249]
[ 0.99799482]
[ 0.97107631]
[ 0.97413936]
[ 0.49046583]
[ 0.98949665]
[ 0.3513475]
[ 0.22559982]
[ 0.98174845]
[ 0.33047864]
[ 0.56353048]
[ 0.70648793]
[ 0.11165485

[ 0.51187434]
[ 0.19747832]
[ 0.83977333]
[ 0.85369649]
[ 0.97713503]
[ 0.6821958]
[ 0.29925399]
[ 0.59525069]
[ 0.93576265]
[ 0.12956837]
[ 0.79667863]
[ 0.92296774]
[ 0.21745493]
[ 0.9371778]
[ 0.07370923]
[ 0.10702984]
[ 0.23643183]
[ 0.56778822]
[ 0.87254197]
[ 0.94067583]
[ 0.1237758]
[ 0.29980392]
[ 0.20355887]
[ 0.62422404]
[ 0.34389789]
[ 0.15462295]
[ 0.0860637]
[ 0.64674859]
[ 0.2704051]
[ 0.85949959]
[ 0.39747702]
[ 0.89631821]
[ 0.68127086]
[ 0.33097945]
[ 0.26217403]
[ 0.75257617]
[ 0.8221641]
[ 0.32146414]
[ 0.76916394]
[ 0.3973786]
[ 0.96634613]
[ 0.78435695]
[ 0.56441257]
[ 0.87251753]
[ 0.6583764]
[ 0.81592871]
[ 0.66949682]
[ 0.61696262]
[ 0.32050294]
[ 0.20788019]
[ 0.06386272]
[ 0.5820201]
[ 0.20708352]
[ 0.43202339]
[ 0.45429728]
[ 0.55432149]
[ 0.25808038]
[ 0.6207252]
[ 0.8357665]
[ 0.20508954]
[ 0.7238888]
[ 0.82282263]
[ 0.21095322]
[ 0.89679536]
[ 0.61542031]
[ 0.80877473]
[ 0.92991114]
[ 0.91563935]
[ 0.29016658]
[ 0.97016756]
[ 0.98965343]
[ 0.79624451]
[ 0.

[ 0.75309796]
[ 0.70849568]
[ 0.13611202]
[ 0.47698606]
[ 0.5470575]
[ 0.75620399]
[ 0.84960999]
[ 0.48250729]
[ 0.69649825]
[ 0.63445756]
[ 0.58676949]
[ 0.86147692]
[ 0.66326774]
[ 0.3892]
[ 0.69829336]
[ 0.02216986]
[ 0.66589991]
[ 0.81854645]
[ 0.52733634]
[ 0.06817105]
[ 0.49871201]
[ 0.75751253]
[ 0.81855964]
[ 0.12086192]
[ 0.22348993]
[ 0.20999696]
[ 0.72235857]
[ 0.6001266]
[ 0.82627869]
[ 0.23656592]
[ 0.2507488]
[ 0.48193103]
[ 0.69384092]
[ 0.86720124]
[ 0.34831589]
[ 0.98236432]
[ 0.36727824]
[ 0.67184045]
[ 0.33733277]
[ 0.31365484]
[ 0.64942266]
[ 0.03498284]
[ 0.16395864]
[ 0.54065061]
[ 0.94726041]
[ 0.85631293]
[ 0.66190176]
[ 0.13327234]
[ 0.04872159]
[ 0.36329307]
[ 0.29805431]
[ 0.28595864]
[ 0.61180685]
[ 0.50936481]
[ 0.82603728]
[ 0.49432137]
[ 0.37296774]
[ 0.53871871]
[ 0.83353616]
[ 0.0327176]
[ 0.00083855]
[ 0.25460677]
[ 0.37547749]
[ 0.19916938]
[ 0.07593575]
[ 0.45395662]
[ 0.30083154]
[ 0.89588761]
[ 0.57196453]
[ 0.32572089]
[ 0.07792808]
[ 0.34885484]


[ 0.69931659]
[ 0.89957047]
[ 0.5080017]
[ 0.58527951]
[ 0.9825654]
[ 0.20141211]
[ 0.73872951]
[ 0.43784794]
[ 0.21141701]
[ 0.3668848]
[ 0.8456993]
[ 0.02209629]
[ 0.06577834]
[ 0.75506564]
[ 0.4364562]
[ 0.51801211]
[ 0.48777861]
[ 0.72710783]
[ 0.1268974]
[ 0.07351828]
[ 0.97912492]
[ 0.39298965]
[ 0.95081259]
[ 0.51704656]
[ 0.95271317]
[ 0.22632668]
[ 0.89713149]
[ 0.5101077]
[ 0.61719196]
[ 0.73379732]
[ 0.67225304]
[ 0.70106238]
[ 0.19050917]
[ 0.47147091]
[ 0.28473669]
[ 0.26791774]
[ 0.62196909]
[ 0.59052755]
[ 0.58427497]
[ 0.12475192]
[ 0.64359797]
[ 0.02378793]
[ 0.61045492]
[ 0.46105298]
[ 0.02703277]
[ 0.91065486]
[ 0.32499493]
[ 0.74766759]
[ 0.88972922]
[ 0.59423041]
[ 0.10585139]
[ 0.46683048]
[ 0.56662039]
[ 0.94813214]
[ 0.10764823]
[ 0.93200881]
[ 0.05169591]
[ 0.43386408]
[ 0.73558487]
[ 0.47205111]
[ 0.60846969]
[ 0.50901255]
[ 0.32130794]
[ 0.01927119]
[ 0.85197582]
[ 0.55707094]
[ 0.26085403]
[ 0.57954203]
[ 0.44234836]
[ 0.50859439]
[ 0.99860594]
[ 0.62129171]

...

[ 0.98014799]
[ 0.54192329]
[ 0.245123]
[ 0.85020194]
[ 0.13802438]
[ 0.64920786]
[ 0.21076746]
[ 0.76718263]
[ 0.03744344]
[ 0.43642083]
[ 0.17142842]
[ 0.36351671]
[ 0.63569628]
[ 0.22332221]
[ 0.01859222]
[ 0.61453935]
[ 0.24454231]
[ 0.59806534]
[ 0.34470588]
[ 0.53440291]
[ 0.61736171]
[ 0.84146857]
[ 0.46646795]
[ 0.92712699]
[ 0.19131657]
[ 0.61334727]
[ 0.26626629]
[ 0.81948122]
[ 0.12018908]
[ 0.56479365]
[ 0.58579636]
[ 0.41548631]
[ 0.55636354]
[ 0.3420184]
[ 0.39112973]
[ 0.72251563]
[ 0.03310887]
[ 0.26286111]
[ 0.71689482]
[ 0.53264715]
[ 0.21936545]
[ 0.43625001]
[ 0.85858185]
[ 0.80187241]
[ 0.22044228]
[ 0.81108347]
[ 0.9612271]
[ 0.14514437]
[ 0.89989567]
[ 0.54367676]
[ 0.95194122]
[ 0.37904811]
[ 0.81912]
[ 0.3944746]
[ 0.16705282]
[ 0.19815311]
[ 0.3033286]
[ 0.85560959]
[ 0.79241599]
[ 0.06664944]
[ 0.11499466]
[ 0.96597461]
[ 0.97609492]
[ 0.90915575]
[ 0.40234314]
[ 0.24244734]
[ 0.41686473]
[ 0.98222108]
[ 0.67555488]
[ 0.42159946]
[ 0.92396955]
[ 0.6325353]
[ 

[ 0.7716177]
[ 0.45538535]
[ 0.87216343]
[ 0.01449161]
[ 0.93658074]
[ 0.59306211]
[ 0.69157238]
[ 0.89115934]
[ 0.97127421]
[ 0.18693198]
[ 0.86393691]
[ 0.16440333]
[ 0.64574251]
[ 0.0624632]
[ 0.06560324]
[ 0.89341781]
[ 0.09098295]
[ 0.71274183]
[ 0.66348134]
[ 0.46089509]
[ 0.18615946]
[ 0.67643966]
[ 0.6446579]
[ 0.88236393]
[ 0.41360087]
[ 0.17325805]
[ 0.16752775]
[ 0.4180959]
[ 0.41131729]
[ 0.44563467]
[ 0.38240053]
[ 0.80463938]
[ 0.68571238]
[ 0.02134315]
[ 0.74330126]
[ 0.03963897]
[ 0.10192058]
[ 0.3451824]
[ 0.98811993]
[ 0.90626358]
[ 0.22353776]
[ 0.54437764]
[ 0.74561788]
[ 0.85262028]
[ 0.80522789]
[ 0.43355716]
[ 0.37921047]
[ 0.10592962]
[ 0.46806944]
[ 0.5037207]
[ 0.30142581]
[ 0.69353072]
[ 0.94395044]
[ 0.83632218]
[ 0.13471834]
[ 0.05779381]
[ 0.28236541]
[ 0.83785873]
[ 0.3920899]
[ 0.59505706]
[ 0.96550455]
[ 0.43150324]
[ 0.07765282]
[ 0.23807355]
[ 0.34080954]
[ 0.51887562]
[ 0.34560101]
[ 0.04263362]
[ 0.7162788]
[ 0.94850125]
[ 0.44430365]
[ 0.90580817]


[ 0.84613528]
[ 0.74366088]
[ 0.85095121]
[ 0.48815338]
[ 0.7798099]
[ 0.14980334]
[ 0.94005655]
[ 0.31431136]
[ 0.31724724]
[ 0.56909367]
[ 0.75064311]
[ 0.93795464]
[ 0.06993888]
[ 0.92043726]
[ 0.54848509]
[ 0.51467298]
[ 0.10250858]
[ 0.18392368]
[ 0.65250093]
[ 0.5532512]
[ 0.07204505]
[ 0.0783632]
[ 0.85819818]
[ 0.70430237]
[ 0.86886637]
[ 0.26342922]
[ 0.50139077]
[ 0.22520947]
[ 0.76233569]
[ 0.27989633]
[ 0.38619929]
[ 0.76059359]
[ 0.89233216]
[ 0.36579352]
[ 0.51375129]
[ 0.43198561]
[ 0.81525196]
[ 0.66471065]
[ 0.76130855]
[ 0.58649906]
[ 0.18386246]
[ 0.41613094]
[ 0.26366827]
[ 0.43629564]
[ 0.84389589]
[ 0.13140718]
[ 0.40079593]
[ 0.84573607]
[ 0.66271957]
[ 0.6799223]
[ 0.59262199]
[ 0.161237]
[ 0.81166981]
[ 0.75869123]
[ 0.01913173]
[ 0.1548623]
[ 0.82703312]
[ 0.25764994]
[ 0.87665133]
[ 0.26119777]
[ 0.2685304]
[ 0.65015067]
[ 0.70113441]
[ 0.4144148]
[ 0.32964808]
[ 0.28551087]
[ 0.52259263]
[ 0.42500026]
[ 0.387862]
[ 0.86099963]
[ 0.01846709]
[ 0.28990953]
[ 0

[ 0.09328733]
[ 0.60552188]
[ 0.48882043]
[ 0.84214441]
[ 0.76149557]
[ 0.82898423]
[ 0.92611089]
[ 0.29980334]
[ 0.31559662]
[ 0.69980604]
[ 0.59126425]
[ 0.57543977]
[ 0.7992611]
[ 0.76119244]
[ 0.53694536]
[ 0.64939797]
[ 0.184249]
[ 0.14125221]
[ 0.2455102]
[ 0.22086029]
[ 0.68432399]
[ 0.28918807]
[ 0.68784597]
[ 0.15583947]
[ 0.45832446]
[ 0.58062105]
[ 0.76827016]
[ 0.42907636]
[ 0.0291243]
[ 0.93119314]
[ 0.51142625]
[ 0.10971809]
[ 0.38645722]
[ 0.64982252]
[ 0.56353301]
[ 0.66377327]
[ 0.44051702]
[ 0.8989591]
[ 0.56178753]
[ 0.2219685]
[ 0.8534087]
[ 0.198105]
[ 0.2464791]
[ 0.64818626]
[ 0.52017796]
[ 0.07110873]
[ 0.78598521]
[ 0.15236224]
[ 0.61311552]
[ 0.90211296]
[ 0.3703462]
[ 0.65290076]
[ 0.01612131]
[ 0.47839801]
[ 0.09550725]
[ 0.33213879]
[ 0.195011]
[ 0.17271711]
[ 0.86585339]
[ 0.98183536]
[ 0.48636826]
[ 0.92798512]
[ 0.75945531]
[ 0.68715323]
[ 0.73251596]
[ 0.64541742]
[ 0.12235592]
[ 0.83100845]
[ 0.59824096]
[ 0.19486232]
[ 0.59947201]
[ 0.809367]
[ 0.8823

[ 0.3524817]
[ 0.79916566]
[ 0.21694108]
[ 0.0912117]
[ 0.22981486]
[ 0.29735125]
[ 0.71104785]
[ 0.12870973]
[ 0.91056055]
[ 0.72677683]
[ 0.47116733]
[ 0.16673049]
[ 0.60845647]
[ 0.1533474]
[ 0.26832476]
[ 0.24880927]
[ 0.70906192]
[ 0.35599999]
[ 0.04878669]
[ 0.56401069]
[ 0.78598883]
[ 0.36482565]
[ 0.4985383]
[ 0.21994052]
[ 0.26183341]
[ 0.82146124]
[ 0.32937784]
[ 0.48235373]
[ 0.84826202]
[ 0.12737359]
[ 0.15637235]
[ 0.20641416]
[ 0.66125044]
[ 0.5262105]
[ 0.03149282]
[ 0.18878135]
[ 0.62885539]
[ 0.41622462]
[ 0.63959886]
[ 0.9792113]
[ 0.66018948]
[ 0.2911571]
[ 0.10492859]
[ 0.71272605]
[ 0.65506229]
[ 0.44710164]
[ 0.86103506]
[ 0.64177299]
[ 0.24435198]
[ 0.58499834]
[ 0.90295819]
[ 0.13339861]
[ 0.59624828]
[ 0.58165637]
[ 0.86514095]
[ 0.44013912]
[ 0.88576099]
[ 0.57409294]
[ 0.05563224]
[ 0.0907543]
[ 0.75603845]
[ 0.29312427]
[ 0.05064488]
[ 0.01588503]
[ 0.78754122]
[ 0.69641235]
[ 0.16402746]
[ 0.94194404]
[ 0.89098574]
[ 0.31739981]
[ 0.12921297]
[ 0.04764671]
